# Cleaning, munging and reshaping data

In [1]:
import pandas as pd
import numpy as np
import os
from datetime import timedelta

In [2]:
os.chdir('/Users/marleneguraieb/Google Drive/Healthcare_metrics')

The first task is to load the dataset containing all the lab files. The dataset contains 4 columns:
    - Medical_unit: the unit that the lab request came from
    - Patient: the unique identifyer of a patient
    - Date_Request: the date the test was requested from the medical unit (this will be the consult date on the consults file)
    - Turnaround: the time it took to process the test, send the results and update the database (this will be relevant as we will have to 'cut' the database to make appropriate comparisons with previous years).

In [3]:
CACU = pd.read_csv('DATA/CACUJul9.csv')

In [4]:
colsEng = ['Medical_unit','Patient','Date_Request','Turnaround']
CACU.columns = colsEng

In [5]:
CACU.Date_Request = pd.to_datetime(CACU.Date_Request,format = '%Y-%m-%d')
CACU.Medical_unit = CACU.Medical_unit.str.replace('NIS','UMF')

In order to make the appropriate comparisons with the previous years, we need to remove the samples in the data that 'took too long' to process in all years. The reason for this is that for each unit, we only have data for x days after the treatment date. Thus, all the tests that take longer than x will not be in the sample of the last year, and since this is the year we're interested in this will bias our results. Thus, for each clinic, we will find out how many days of data we have (as in last capture date minus treatment date) and we will remove from all previous years all the tests that took longer to process than these. 

In [6]:
#Get the last capture date per medical unit in the dataset
CACU['Capture'] = CACU['Date_Request'] + pd.to_timedelta(CACU['Turnaround'], unit = 'd')

MaxCapture = CACU[['Medical_unit','Capture']].groupby('Medical_unit').max()
MaxCapture['Medical_unit'] = MaxCapture.index

In order to know how many 'days of data' we hav eper clinic, we need to know the date of every treatment in the clinics. This is where TMTstats is needed. A dataframe that contains the treatment date and treatment (control or treatment) per medical unit. 

In [7]:
#Now we need to get the relevant treatment dates per clinic in the dataset given the treatment dates. 
TMTstats = pd.read_table('DATA/TMTstats.csv')

colsEng = ['index','Medical_unit','Treatment','TMTdate']
TMTstats.columns = colsEng
TMTstats.TMTdate = pd.to_datetime(TMTstats.TMTdate,format = '%Y-%m-%d')

TMTstats = pd.merge(TMTstats[['Medical_unit','Treatment','TMTdate']], 
                    MaxCapture, how='left')

TMTstats['MaxTurnaround'] = TMTstats['Capture']-(TMTstats['TMTdate']+pd.to_timedelta(30,unit = 'd'))


Once we have this, we will only look at tests (across all years) that took less than our "data window" in the treatment year

In [8]:
CACUcut = pd.merge(CACU,TMTstats[['Medical_unit','MaxTurnaround']],how='left')

#Fill the clinics that are not in the TMTstats database with the mean
CACUcut['MaxTurnaround']= CACUcut['MaxTurnaround'].fillna(TMTstats['MaxTurnaround'].mean())

CACUcut.Turnaround = pd.to_timedelta(CACUcut.Turnaround, unit = 'D')

CACUcut = CACUcut[CACUcut.Turnaround<=CACUcut.MaxTurnaround]

#Now only the clinics we want:
CACUcut = CACUcut[CACUcut.Medical_unit.isin(set(TMTstats.Medical_unit))]


Now we need to get the data in shape. The lab files are ``stacked``, that is, they contain every test that the lab made, including more than one test per patient. What we need is patient histories, a dataframe indexed by (unique) patient that lists, in the columns all the dates that patients got the tests. 

In [9]:
## Now we need to see by patients, who got a test and how many times in order to
#construct patient histories. Missing values are all zeroes or all 8's, so let's
#get rid of them:
    
CACUcut = CACUcut[CACUcut.Patient.str.contains('8888888888|0000000000') == False]

#Now let's make a unique patient database with all the tests each patient got
# every column is a new test, with the date. 

CACUcut = CACUcut.sort_values(['Patient','Date_Request'],ascending=True,axis=0)

list = CACUcut.groupby('Patient').apply(lambda x: range(len(x))).tolist()
loop = [val for sublist in list for val in sublist]
CACUcut['obs']=loop


CACUwide = CACUcut[['Patient','Date_Request','obs']].pivot(index='Patient',
                 columns= 'obs', values='Date_Request')


In [10]:
colsEng = [str(x) for x in range(0,9)]
colsEng = ['Date.' + s for s in colsEng]

CACUwide.columns = colsEng


# Determining Eligibility

The rule for determining eligibility of the test is the following:

    -Every woman between the ages of 25 and 65 (the whole database) needs one test every year. 
    -Except if a woman has had two consecutive negative tests in the two previous years she can go 3 years without the test. 
    -We can determine this from the lab files (without looking at doctor visits), by creating a window of time that every patient is covered after a test, and compare it to when she got the tests from her histories. 
    - We will code this as 'HT' for when a woman got a test she needed, and 'HNT' for when she got a test she did not need. 

In [11]:
colsEng = [str(x) for x in range(0,9)]
colsEng = ['Elegible.' + s for s in colsEng]

CACUwide = pd.concat([CACUwide,pd.DataFrame(columns=colsEng)])

#The first one, since there is no previous one, is always needed
CACUwide['Elegible.0'] = 'HT'

CACUwide['B_Cper.0'] = CACUwide.iloc[:,0]
CACUwide['E_Cper.0'] = CACUwide.iloc[:,0]+pd.to_timedelta(365,unit ='D')


In [12]:
for i in range(0,8):
#first find the relevant columns for each date
    cols = CACUwide.columns.values.tolist()
    cols = [cols.index(s) for s in cols if str(i) in s]
#then, based on the period covered determine if that date was within covered period (HNT) or not (HT)
    CACUwide.iloc[:,cols[1]+1] = np.where(CACUwide.iloc[:,cols[0]+1]<=CACUwide.iloc[:,cols[3]],
    'HNT','HT')
    CACUwide.iloc[:,cols[1]+1] = np.where(pd.isnull(CACUwide.iloc[:,cols[0]+1])==True,
    CACUwide.iloc[:,cols[0]+1],CACUwide.iloc[:,cols[1]+1])
#then determine new covered period
#first, if test was done but not needed, just add 365 to E_Pc
    CACUwide.iloc[:,cols[3]]= np.where(CACUwide.iloc[:,cols[1]+1]=='HNT',
    CACUwide.iloc[:,cols[0]+1]+pd.to_timedelta(365,unit='D'),CACUwide.iloc[:,cols[3]])
#Then create new columns for Beginning and end of coverage period
    CACUwide['B_Cper.' + str(i+1)] = pd.to_datetime('nan')
    CACUwide['E_Cper.' + str(i+1)] = pd.to_datetime('nan')
#Then fill them with the date, and the date+365 if the patient got the test
#the beginning of the coverage period is always the date request
    CACUwide.iloc[:,cols[3]+1] = np.where(CACUwide.iloc[:,cols[1]+1]=='HT',
    CACUwide.iloc[:,cols[0]+1],CACUwide.iloc[:,cols[3]+1])
#the end is more complicated. there are three cases if she's had two in the past years:
    CACUwide.iloc[:,cols[3]+2] = np.where((CACUwide.iloc[:,cols[1]+1]=='HT') &
    (CACUwide.iloc[:,cols[0]+1]-CACUwide.iloc[:,cols[0]]<pd.to_timedelta(400,unit='D')),
    CACUwide.iloc[:,cols[0]+1]+pd.to_timedelta(1095,unit ='D'),CACUwide.iloc[:,cols[3]+1])
#if she hasn't she needs another one next year
    CACUwide.iloc[:,cols[3]+2] = np.where((CACUwide.iloc[:,cols[1]+1]=='HT') &
    (CACUwide.iloc[:,cols[0]+1]-CACUwide.iloc[:,cols[0]]>=pd.to_timedelta(400,unit='D')),
    CACUwide.iloc[:,cols[0]+1]+pd.to_timedelta(365,unit ='D'),CACUwide.iloc[:,cols[3]+1])
#and if she didn't need it she just gets +365
    CACUwide.iloc[:,cols[3]+2] = np.where(CACUwide.iloc[:,cols[1]+1]=='HNT',
    CACUwide.iloc[:,cols[0]+1]+pd.to_timedelta(365,unit ='D'),CACUwide.iloc[:,cols[3]+1])
#and then the ones we didn't categorize we just import the old coverage period
    CACUwide.iloc[:,cols[3]+1] = np.where((pd.isnull(CACUwide.iloc[:,cols[1]+1])==False) &
    (pd.isnull(CACUwide.iloc[:,cols[3]+1])==True),CACUwide.iloc[:,cols[2]],CACUwide.iloc[:,cols[3]+1])
    CACUwide.iloc[:,cols[3]+2] = np.where((pd.isnull(CACUwide.iloc[:,cols[1]+1])==False) &
    (pd.isnull(CACUwide.iloc[:,cols[3]+2])==True),CACUwide.iloc[:,cols[3]],CACUwide.iloc[:,cols[3]+2])


This gives us a dataframe that is:
    a) unique by patient 
    b) contains a list (in columns) of the test each patient got
    c) for each test, can tell us whether the patient needed the test or not
    d) contains windows of coverage (indexed from B_Cper.0 to B_Cper.8 and E_Cper.0 to E_Cper.8) that represent the time intervals in the dataframe where each patient was 'covered', that is, a doctor visit between B_Cper.x and E_Cper.x does not need a pap spear. 
    
The next step will be to merge this dataframe with the doctor visits. 

In [14]:
CACUwide.to_csv('CACUwide.csv')